**Data Visualization course - winter semester 2023/24 - FU Berlin**

*Tutorials adapted from the [Information Visualization](https://infovis.fh-potsdam.de/tutorials/) course at the FH Potsdam*

# Tutorial 2: Data wrangling

Welcome back! This tutorial shows you a few tricks for preparing data for visualization. You will see how data can be loaded, parsed, and examined. For this we will continue to work with the **Pandas** package, in particular with the DataFrame data structure, and get to know a few additional helpers. 

*Just as in the first tutorial, you should be able to run the notebook yourself, edit the contents of all cells, and try things out. Here and there particular opportunities for such edits are highlighted with a pencil.*

In [ ]:
import pandas as pd
import altair as alt

## Loading 

The first step is to bring the data into the purview of your notebook. So regardless of data structure and format, you need to have access to the data set. We will briefly cover four common ways of loading data into your Jupyter notebook.

### Enter data directly

The simplest way to add data to your notebook is to enter it verbatim into the notebook as we have seen with the capital cities in the first tutorial:

In [ ]:
cities = pd.DataFrame({
  "name": ["Athens", "Bratislava", "Copenhagen", "Dublin"],
  "area": [39, 367.6, 86.2, 115],
  "elevation": [170, 152, 14, 20],
  "population": [664046, 429564, 602481, 553165]
  }
)

cities

✏️ *Add a column for years when you have visited or plan to visit these cities. You can made up some years, if needed.*

### Open a local file

You might also want to open a local file. We can open a file using Python's built-in `Zipfile` method in a context manager, after which we can `read()` its contents into the variable `covid_json`. The file is automatically closed when the context manager block is left. In this case the data is in the JSON format, which we will need to parse. We'll get to this later. You can open all kinds of formats. Here we know that we are dealing with a JSON file because of its extension.



In [ ]:
from zipfile import ZipFile

with ZipFile("owid-covid-data.zip", 'r') as file:
    covid_json = file.read('owid-covid-data.json')

covid_json

### Get data via a URL

There are some methods that can directly load a dataset via a URL, i.e., a web address. For others you might have to retrieve the file first to continue parsing it. The `requests` package helps you to send HTTP requests and retrieve the responses. 

In the following, the news feed of Tagesschau is retrieved via an HTTP GET request. Note that the news feed is made available as an XML format; of course you can retrieve all kinds of file formats using this method:

In [ ]:
import requests

response = requests.get('https://www.tagesschau.de/infoservices/alle-meldungen-100~rss2.xml')
response
tagesschau_xml = response.text
tagesschau_xml[:1000] # this displays the first 100 characters

Please note: To display the data from the XML response in a more structured and readable format, you can use an XML parsing library like xml.etree.ElementTree. 

✏️ *Find the news feed for another webpage (e.g., RSS feed from the BBC News website using the URL 'http://feeds.bbci.co.uk/news/rss.xml) and try to load it*

### Use an API

Some web platforms require the use of an API (application programming interface) to get access to their data. Simply put, an API is a structured way to request and retrieve data. Oftentimes it is just a specific way to format the URL.

The German National Library offers an [API to query Entity Facts](https://www.dnb.de/EN/Professionell/Metadatendienste/Datenbezug/Entity-Facts/entityFacts_node.html) contained in the GND (Gemeinsame Normdatei). In this case the API provides the data in the JSON format, which has become quite common for web APIs, but you will also encounter many other formats. 

To retrieve information for a given GND entity by its id, such as the GND entry for the artist [Käthe Kollwitz](https://en.wikipedia.org/wiki/Käthe_Kollwitz) you have to put the `base_url` together with the `gnd_id`: 

In [ ]:
base_url = "https://hub.culturegraph.org/entityfacts/"
gnd_id = "118564943" # GND identifier from the Wikipedia page on Käthe Kollwitz
gnd_response = requests.get(base_url+gnd_id).json()

gnd_response

Using such API is very helpful, for example, when adding additional data to an existing data set. Be aware, very often API keys are needed to access APIs. You need to request them first. An example is the OpenWeatherMap API and how to request data with an API key is shown in the following example:

In [ ]:
# Replace 'your_api_key' with your actual API key
api_key = 'your_api_key'

# Specify the city for which you want weather data
city = 'London'

# Create the API URL
api_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

✏️ *Prepare an API request with a GND id of another person of German history*

## Parsing 

Apart from our little cities example, so far we have only loaded the data into unstructured strings. To be able to analyze the data, we have to turn the unstructured strings of symbols into a practical data structure of the DataFrame that we can work with. This process is commonly referred to as ‘parsing’. 

As we have seen above, data can come in various file formats, which are in turn more or less appropriate for particular data structures. We'll cover four typical ones in the following section, but we will see more over the course of the tutorials to come.

The different ways of loading data (e.g., by file path or URL) are independent from the particular data formats provided. For example, you can load CSV data from a local file or from a web address. While the files typically indicate with the extension what format they have, URLs or APIs may not have these. If it is not clear, you may have to check the documentation or take a peek into the file.

### CSV

The CSV format is probably the most common file format in the context of data analysis and visualization. CSV files contain tabular data that can be viewed and edited in a spreadsheet software such as Excel. CSV stands for [comma-separated values](https://en.wikipedia.org/wiki/Comma-separated_values), which seems to say it all: the data values are separated by commas and each row represents one item. However, there are also CSV files that use separators other than commas, such as tabs and semicolons. 

Let's load a CSV file! Thankfully Pandas has the convenient `read_csv()` method ready for us, which can open CSV data via a file path or URL, and turns it directly into a DataFrame object. 


In [ ]:
covid_data = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")

covid_data.head()

✏️ *Try loading another CSV dataset from [Berlin's Open Data Portal](https://daten.berlin.de/)*

### XML

[XML](https://en.wikipedia.org/wiki/XML) (Extensible Markup Language) is a data format, which can have very different kinds of hierarchical structures. XML files are common in a wide variety of contexts, including libraries, and especially in situations, in which the interoperability of multiple systems by several vendors needs to be ensured.

The 🌲 **ElementTree** module will help us to parse the elements contained in an XML file. 

In [ ]:
import xml.etree.ElementTree as ET

As we have already retrieved the XML feed from Tagesschau (and saved it in the variable `tagesschau_xml`), we can now parse it directly from the string, i.e., using the method `ET.fromstring()`:

In [ ]:
tagesschau = ET.fromstring(tagesschau_xml)
tagesschau

This gives the root element of the XML feed (and all its children) in the variable `tagesschau`. 

Going through all items with `findall` and within these with `find` for specific sub-elements, we can extract the publication date and time and the title of the respective item. In the following these elements are put together into the DataFrame `tagesschau_df`. Note that it helps to peek into the XML source of the feed to know the specific element names.

In [ ]:
# create two empty lists
dates = []
titles = []

# go through all item elements in the tree
for item in tagesschau.findall('.//item'):
  # extract date information and titles
  dates.append( item.find('.//pubDate').text )
  titles.append( item.find('.//title').text )

# create a dataframe containing the two columns
tagesschau_df = pd.DataFrame(    
    {'date': dates,
     'title': titles,
    })  

tagesschau_df

✏️ *Each news item also contains a `description` element. Why not add a third column to the DataFrame?*

## Examining

You are now able to load and parse data from several formats. At this point, here are plenty of ways to inspect these datasets. We are going to try some simple methods to peek around the datasets. Once you have a tabular dataset ready as a DataFrame, there are quite a few convenient methods to view and explore its contents.

### Head & tail

You could start with looking at the beginning of the dataset with `head()`:

In [ ]:
covid_data.head()

✏️ *What do you think happens, when you replace `head()` with `tail()` ?*

### Describe & info

You can also ask Pandas to provide some statistical descriptions (which are only applied to the columns containing numeric data):


In [ ]:
covid_data.describe()

This may not seem that useful to you yet. You may want to know what kind of datatypes the different columns contain and how many values are present. For this the `info()` method will be of help:

In [ ]:
covid_data.info()

With this it is now possible to access specific columns by using their names.  But did you notice the long label for the first column? Let's rename the column `human_development_index` into something short and sweet such as: `hdi`:

In [ ]:
covid_data = covid_data.rename(columns={"human_development_index": "hdi"})

✏️ *Do you want to rename any other columns?*

### Select & query

We can select an individual column using single [square brackets]:

In [ ]:
covid_data["hdi"]

… and we can select multiple columns using nested [[square brackets]]:

In [ ]:
covid_data[["hdi", "life_expectancy"]]

✏️ *Which columns interest you? Replace `hdi` and `life_expectancy` with other column labels* 

Note that the `life_expectancy` columns contains a lot of `NaN` - this stands for "Not a Number" and it means here that values are missing.

In order to focus on the rows which do have missing data, we can squeeze in a requirement that we only want those rows, where the values in both above used columns are not missing, i.e., `notnull()`:

✏️ *Formulate a query on another column:*

There are four related methods for accessing rows, columns, and specific values, either by integer positons (iloc and iat) or by the labels (that is what is displayed in bold above).

- `loc`: access rows and columns by label
- `iloc`: access rows and columns by integer position 
- `at`: access a single value for a row/column label pair
- `iat`: access a single value for a row/column pair by integer position 

For example, this way we can get the first entry in the `covid_data` DataFrame: 

In [ ]:
covid_data.loc[0]

# because the index here uses integers, iloc and loc do the same

Finally, you can also retrieve rows that match a query. With this we are retrieving the rows with new cases for Germany:

In [ ]:
covid_data[covid_data['location'].str.contains('Germany')] ["new_cases"]

Let's assume we have a dataframe like this:

In [ ]:
import numpy as np

data = pd.DataFrame({
 'day': list(range(100)),
 'Hagworts': np.random.randint(0, 500, 100).cumsum(), 
 'William Wunder\'s Factory' : np.random.randint(0, 100, 100).cumsum(),
 'Lotham' : np.random.randint(0, 1000, 100).cumsum()   
})

data.head()

If we want to show that in the form of a line graph, we have one Problem! Which column do we select as the y axis???

Fortunately, Pandas comes to help! Using ```melt``` we can merge multiple columns into one.

In [ ]:
merged_data = data.melt('day')
merged_data

In [ ]:
alt.Chart(merged_data).mark_line().encode(
    x = 'day',
    y = 'value',
    color = 'variable')

✏️ *Now it's time for a challenge!*

Try to visualize the positive cases per million over time for three countries of your choosing! There are several steps to it: First you need to convert the date column to a pandas date column using ```pd.to_datetime```, then you filter by the countries and finally you can visualize it.

## Sources
- [Pandas Tutorial: DataFrames in Python - DataCamp](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
- [The ElementTree XML API](https://docs.python.org/2/library/xml.etree.elementtree.html)
- [Where do Mayors Come From? Querying Wikidata with Python and SPARQL - Towards Data Science](https://towardsdatascience.com/where-do-mayors-come-from-querying-wikidata-with-python-and-sparql-91f3c0af22e2)
- [Examining Data Using Pandas | Linux Journal](https://www.linuxjournal.com/content/examining-data-using-pandas)
- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)